## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

2023/07/12 20:05:46 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/07/12 20:05:46 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


AttributeError: 'MlflowClient' object has no attribute 'list_experiments'

In [3]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [4]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [12]:
for run in runs:
    print(f"run id: {run.info.run_id},duration time: {run.info.end_time - run.info.start_time}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 5afe220a34cb453786e6fb6914e56c6c,duration time: 177590, rmse: 6.3146
run id: 5bee67d0744c4693ba4e8b7243824bec,duration time: 84959, rmse: 6.3210
run id: b5a1d43d60e8468ea824b9ced0e7eecc,duration time: 74138, rmse: 6.3426
run id: 9a30bb5961244d0481fbf8f55bba8d27,duration time: 130024, rmse: 6.3632
run id: bfa1452468e044f1927bc4294f63b418,duration time: 42175, rmse: 6.3956


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [6]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [13]:
run_id = "5bee67d0744c4693ba4e8b7243824bec"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="XGBoost")

2023/07/12 20:12:21 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/07/12 20:12:21 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Registered model 'XGBoost' already exists. Creating a new version of this model...
2023/07/12 20:12:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: XGBoost, version 5
Created version '5' of model 'XGBoost'.


<ModelVersion: aliases=[], creation_timestamp=1689185541557, current_stage='None', description=None, last_updated_timestamp=1689185541557, name='XGBoost', run_id='5bee67d0744c4693ba4e8b7243824bec', run_link=None, source='/Users/daniloneves/Documents/Files/Projects/Git/mlops-mlflow/1_experiment_tracking/mlruns/1/5bee67d0744c4693ba4e8b7243824bec/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

In [16]:
model_name = "XGBoost"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 4, stage: Staging
version: 3, stage: Archived
version: 5, stage: None


In [17]:
model_version = 4
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1689184156414, current_stage='Staging', description='', last_updated_timestamp=1689185672629, name='XGBoost', run_id='5afe220a34cb453786e6fb6914e56c6c', run_link='', source='/Users/daniloneves/Documents/Files/Projects/Git/mlops-mlflow/1_experiment_tracking/mlruns/1/5afe220a34cb453786e6fb6914e56c6c/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [18]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1689184156414, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2023-07-12', last_updated_timestamp=1689185676521, name='XGBoost', run_id='5afe220a34cb453786e6fb6914e56c6c', run_link='', source='/Users/daniloneves/Documents/Files/Projects/Git/mlops-mlflow/1_experiment_tracking/mlruns/1/5afe220a34cb453786e6fb6914e56c6c/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=4>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [19]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [21]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [ ]:
client.download_artifacts(run_id=run_id, path='1_experiment_tracking/preprocessor/preprocessor.b')

In [27]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [28]:
X_test = preprocess(df, dv)

In [29]:
target = "duration"
y_test = df[target].values

In [34]:
model_version = 1
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1689176013434, current_stage='Production', description='', last_updated_timestamp=1689186002884, name='XGBoost', run_id='bfa1452468e044f1927bc4294f63b418', run_link='', source='/Users/daniloneves/Documents/Files/Projects/Git/mlops-mlflow/1_experiment_tracking/mlruns/1/bfa1452468e044f1927bc4294f63b418/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [35]:
model_version = 4
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1689184156414, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2023-07-12', last_updated_timestamp=1689186004665, name='XGBoost', run_id='5afe220a34cb453786e6fb6914e56c6c', run_link='', source='/Users/daniloneves/Documents/Files/Projects/Git/mlops-mlflow/1_experiment_tracking/mlruns/1/5afe220a34cb453786e6fb6914e56c6c/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [36]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

2023/07/12 20:20:06 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.1, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[20:20:06] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 7.47 s, sys: 892 ms, total: 8.37 s
Wall time: 1.62 s


{'rmse': 6.329582178646738}

In [37]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

2023/07/12 20:20:09 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.1, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[20:20:09] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 22.5 s, sys: 2.1 s, total: 24.6 s
Wall time: 4.5 s


{'rmse': 6.258709554165533}

In [38]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1689184156414, current_stage='Production', description='The model version 4 was transitioned to Staging on 2023-07-12', last_updated_timestamp=1689186039040, name='XGBoost', run_id='5afe220a34cb453786e6fb6914e56c6c', run_link='', source='/Users/daniloneves/Documents/Files/Projects/Git/mlops-mlflow/1_experiment_tracking/mlruns/1/5afe220a34cb453786e6fb6914e56c6c/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=4>